In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import matplotlib.pyplot as plt
from model import ConditionalDenoiser, FiLM, CrossAttention, SinusoidalTimeEmbedding

In [2]:
model_name = "Two_stage_FiLM_Diffusion"
schedule_name = "Sigmoid"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
print("[INFO] Loading Two-Stage FiLM model checkpoint...")
# Allowlist your custom class for safe unpickling
torch.serialization.add_safe_globals([ConditionalDenoiser, FiLM, CrossAttention, SinusoidalTimeEmbedding])

# === Load checkpoint ===
checkpoint_path = "Two_stage_FiLM_Diffusion_checkpoint_Sigmoid.pth"
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

# === Recreate model ===
T = checkpoint["T"]
latent_dim = 256
time_dim = 64
label_emb_dim = 64
n_classes = checkpoint["model"].n_classes

model = ConditionalDenoiser(
    T=T,
    latent_dim=latent_dim,
    n_classes=n_classes,
    time_dim=time_dim,
    label_emb_dim=label_emb_dim,
    classifier_free_prob=0.0
).to(device)

# Optional: restore betas if needed
betas = checkpoint["betas"].to(device)
model.betas = betas.to(device)

# === Load trained weights ===
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

print("Model loaded successfully for inference!")

In [ ]:
print("[INFO] Loading test embeddings (z_test.npy, y_test.npy)...")
z_test = torch.tensor(np.load("z_test.npy"), dtype=torch.float32).to(device)
y_test = torch.tensor(np.load("y_test.npy"), dtype=torch.long).to(device)

In [ ]:
# ============================================
# Map labels to contiguous [0..num_classes-1]
# ============================================
unique_labels = torch.unique(y_test)
label_map = {int(v): i for i, v in enumerate(unique_labels)}
num_classes = len(unique_labels)

y_test_idx = torch.tensor(
    [label_map[int(y)] for y in y_test.cpu().numpy()],
    device=device
)

label_ids = torch.arange(num_classes, device=device)

print(f"[INFO] Found {num_classes} unique classes")

In [7]:
@torch.no_grad()
def softmax_posterior_latent(
    z0_batch, den_model, betas, T,
    n_paths=256, num_classes=10, device="cuda",
    eps_cache=None, class_chunk_size=10, use_autocast=True
):
    
    """
    Memory-efficient posterior inference:
    - Processes classes in chunks instead of expanding [B*num_classes, n_paths, T+1, D].
    - Optional FP16 autocast for lower VRAM usage.

    Returns:
        map_labels: [B]
        posteriors: [B, num_classes]
    """
    B, D = z0_batch.shape
    alphas = 1. - betas
    sqrt_a = torch.sqrt(alphas)
    sqrt_1ma = torch.sqrt(1. - alphas)

    # === Noise path reuse (antithetic or given) ===
    if eps_cache is None:
        eps = torch.randn(n_paths, T, D, device=device)
    else:
        eps = eps_cache.to(device)

    # === Precompute diffusion forward paths ===
    z_path = torch.empty(B, n_paths, T + 1, D, device=device)
    z_path[:, :, 0, :] = z0_batch[:, None, :]
    for t in range(T):
        z_path[:, :, t + 1, :] = sqrt_a[t] * z_path[:, :, t, :] + sqrt_1ma[t] * eps[None, :, t, :]

    # === Prepare log-likelihood storage ===
    ll_mean_all = torch.zeros(B, num_classes, device=device)

    # === Loop through class chunks to avoid [B*num_classes] explosion ===
    for c_start in range(0, num_classes, class_chunk_size):
        c_end = min(c_start + class_chunk_size, num_classes)
        this_chunk = c_end - c_start
        y_label = torch.arange(c_start, c_end, device=device)
        y_label = y_label.repeat_interleave(n_paths * B)

        ll_chunk = torch.zeros(B * this_chunk * n_paths, device=device)

        # === Main likelihood accumulation ===
        autocast_context = torch.autocast(device_type="cuda", dtype=torch.float16) if use_autocast else torch.cuda.amp.autocast(enabled=False)
        with autocast_context:
            for t in range(T):
                zt = z_path[:, :, t, :].repeat(this_chunk, 1, 1).reshape(-1, D)
                zt1 = z_path[:, :, t + 1, :].repeat(this_chunk, 1, 1).reshape(-1, D)
                t_vec = torch.full_like(y_label, t + 1, dtype=torch.long)
                score = den_model(zt1, t_vec, y_label)
                ll_chunk += (score * (zt1 - zt)).sum(dim=1)
                ll_chunk += 0.5 * betas[t] * (-(score**2).sum(dim=1) + (zt1 * score).sum(dim=1))

        # === Reshape back and average across paths ===
        ll_chunk = ll_chunk.view(this_chunk, B, n_paths).permute(1, 0, 2)
        ll_mean_chunk = ll_chunk.mean(dim=2)
        ll_mean_all[:, c_start:c_end] = ll_mean_chunk

        torch.cuda.empty_cache()  # optional, helps prevent fragmentation

    # === Compute posterior and MAP ===
    posterior = torch.softmax(ll_mean_all, dim=1)
    map_labels = posterior.argmax(dim=1)

    return map_labels.cpu().numpy(), posterior.cpu().numpy()

In [8]:
import pickle

print("[INFO] Running per-class posterior plots using Softmax posterior...")

# y_test: original labels (not remapped)
y_test_np = y_test.cpu().numpy()

# original labels in the same order as used to define label_map
unique_test_labels = [int(v) for v in unique_labels.cpu().numpy()]  # list of original label IDs

results = {}
y_preds = []
posteriors = []   # each entry: posterior over contiguous class indices [0..num_classes-1]

for class_idx, true_label in enumerate(unique_test_labels):
    print(f"\n[RUN] Test Class {class_idx} (true label={true_label})")

    # ---- all samples belonging to this original label ----
    class_sample_indices = np.where(y_test_np == true_label)[0]

    if len(class_sample_indices) == 0:
        print(f"[WARN] Class {true_label} has zero samples in y_test.")
        continue

    # ---- mean latent embedding for this class ----
    z0_obs = z_test[class_sample_indices].mean(dim=0).to(device).unsqueeze(0)  # [1, D]

    # ---- compute posterior via softmax_posterior_latent ----
    map_label_idx, posterior = softmax_posterior_latent(
        z0_batch=z0_obs,               # [1, D]
        den_model=model,
        betas=betas,
        T=T,
        n_paths=256,
        num_classes=num_classes,
        device=device,
        class_chunk_size=15,
    )

    post_np = posterior[0]            # shape [num_classes]
    map_label_contig = int(map_label_idx[0])  # index 0..num_classes-1
    # map back to original label space
    pred_label_orig = int(unique_labels[map_label_contig].item())

    print(f"True label {true_label}: MAP contiguous={map_label_contig}, "
          f"MAP original={pred_label_orig}\nPosterior: {post_np}")

    # ---- store for saving later ----
    y_preds.append(pred_label_orig)       # predicted label in original label space
    posteriors.append(post_np)           # posterior over contiguous indices

    results[true_label] = {
        "true_label": true_label,
        "pred_label_contig": map_label_contig,
        "pred_label_orig": pred_label_orig,
        "posterior": post_np,
        "n_cells": len(class_sample_indices),
    }

    # ---- PLOT  ----
    fig, ax = plt.subplots(figsize=(8, 5))
    bars = ax.bar(range(len(post_np)), post_np, color="skyblue", edgecolor="black")

    ax.set_xlabel("Perturbation index (contiguous class idx)", fontsize=12)
    ax.set_ylabel("Posterior probability", fontsize=12)
    ax.set_title(
        f"Posterior — True {true_label}, MAP orig {pred_label_orig} (idx {map_label_contig})",
        fontsize=14
    )

    # top_k annotation
    top_k = 3
    sorted_idx = np.argsort(post_np)[::-1][:top_k]
    for idx in sorted_idx:
        ax.annotate(
            f"{idx}\n{post_np[idx]:.3f}",
            xy=(idx, post_np[idx]),
            xytext=(0, 5),
            textcoords="offset points",
            ha='center', va='bottom', fontsize=6,
            bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="black", lw=0.5)
        )

    ax.set_ylim(0, max(post_np) * 1.18)
    ax.tick_params(axis='x', labelrotation=45)
    fig.tight_layout()
    fig.savefig(f"softmax_{T}_steps_class_{class_idx}_posterior.png", dpi=150)
    plt.close(fig)

print("[INFO] Softmax posterior per-class inference completed.")

# ============================================================
# === SAVE ARRAYS + FULL RESULTS =============================
# ============================================================

y_preds = np.array(y_preds)
posteriors = np.stack(posteriors, axis=0) 

np.save("y_pred_test_softmax.npy", y_preds)
np.save("posterior_test_softmax.npy", posteriors)

with open("softmax_posterior_results.pkl", "wb") as f:
    pickle.dump(results, f)

print("[INFO] Saved:")
print("  - y_pred_test_softmax.npy (shape:", y_preds.shape, ")")
print("  - posterior_test_softmax.npy (shape:", posteriors.shape, ")")
print("  - softmax_posterior_results.pkl")